In [3]:
import numpy as np

In [1]:
#5.5활성화 함수 계층 구현하기
class Relu:                     #렐루 클래스
    def __init__(slef):         #초기화 
        self.mask = None        #마스크 선언 -> 인스턴스 변수.mask로 쓸수있음
        
    def forward(self,x):        #순방향
        self.mask = (x<=0)      #self.mask에 bool형태로 저장
        out = x.copy()          #out에 x넘파이 배열을 복사
        out[self.mask] = 0      #self.mask는 0과 1로 이루어진 배열임. out[self.mask] = 0으로 하니까 self.mask가 1인 애들만 0으로 변함
        
        return out              #0이하인 애들은 0이고 나머지는 y=x인 렐루함수 구현 완료☆
    
    def backward(self, dout):   #역방향
        dout[self.mask] = 0     #dout은 흘러내려오는 값을 0이하인 애들만 0으로 만들기
        dx = dout               
        
        return dx               #렐루 역방향 구현 완료☆


In [6]:
#위코드 보충설명
x = np.array ([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x<=0)           #0보다 작은 값들은 False 
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [9]:
#Sigmoid 계층 구현
class Sigmoid:          
    def __init__(self):
        self.out = None
        
    def forward(self,x):            
        out = 1/(1+np.exp(-x))
        self.out = out              #Q. 그냥 self.out = 1/(1+np.exp(-x))로 하면 되는거 아닌가 왜 구지 out이라는 변수를 만들어서 ...
                                    #A. 파이썬 언어는 포인터로 변수를 벼환하기 떄문
        return out
    
    def backward(self, dout):       
        dx = dout * (1.0 - self.out) * self.out     #시그모이드 함수 미분 결과를 반환
        
        return dx

In [ ]:
#Affine 계층 보충설명
X = np.random.rand(2)               #입력
W = np.random.rand(2,3)             #가중치
B = np.random.rand(3)               #편향

X.shape
W.shape
B.shape

Y = np.dot(X,W) + B

In [4]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

X_dot_W

X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [5]:
dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis=0)
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [21]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x   #입력데이터를 받아온다
        out = np.dot(x, self.W) + self.b    
        
        return out 
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

In [17]:
#Softmax-with-Loss 계층 구현
import sys,os
sys.path.append(os.path.abspath('../../../'))
from common.functions import softmax, cross_entropy_error
class SoftmaxWithloss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [22]:
import sys, os
sys.path.append(os.path.abspath('../../../'))
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        #가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)
        
        #계층생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
            
        self.lastLayer = SoftmaxWithloss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    #x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        #순전파
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        #결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads         

In [23]:
import sys, os
sys.path.append(os.path.abspath('../../../'))
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) =\
    load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

AttributeError: 'NoneType' object has no attribute 'shape'

In [14]:
import sys, os
sys.path.append(os.path.abspath('../../../'))
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 1000
train_size = x_train.shape[0]
batch_size= 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

AttributeError: 'numpy.ndarray' object has no attribute 'nidm'